## Data loading and preprocessing

In [3]:
import os
import numpy as np 
import cv2
import random

In [41]:
path = './License_plate_data_set_2021/'
imgName = os.listdir(path)
minArea = 1000
minPlateRatio = 0.5
maxPlateRatio = 6.0  
#print(imgList)
def img_pro_sobel(imgName):
    img_gaus = cv2.GaussianBlur(imgName,(5,5),0)
    img_gray = cv2.cvtColor(img_gaus,cv2.COLOR_BGR2GRAY)
    
    sobel = cv2.Sobel(img_gray, cv2.CV_8U, 1, 0)
    
    _,img_thresh = cv2.threshold(sobel,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    cv2.imshow("sobel",img_thresh)
    cv2.waitKey(0)
    img_edge = cv2.Canny(img_thresh,100,200)
    # 膨胀和腐蚀操作的核函数
    kernel = np.ones((10, 19), np.uint8)                                                
    imgEdge = cv2.morphologyEx(img_edge, cv2.MORPH_CLOSE, kernel)                          
    imgEdge = cv2.morphologyEx(imgEdge, cv2.MORPH_OPEN, kernel)                          
    cv2.imshow("imgEdgeProcessed", imgEdge) 
    cv2.waitKey(0)
    element1 = cv2.getStructuringElement(cv2.MORPH_RECT, (9, 1))
    element2 = cv2.getStructuringElement(cv2.MORPH_RECT, (8, 6))
    # 膨胀一次，让轮廓突出
    dilation = cv2.dilate(img_thresh, element2, iterations = 1)
    # 腐蚀一次，去掉细节
    erosion = cv2.erode(dilation, element1, iterations = 1)
    # 再次膨胀，让轮廓明显一些
    dilation2 = cv2.dilate(erosion, element2,iterations = 3)
    
    return dilation2

def img_pro_enhance(imgName):
    kernel = np.array([[0, -1, 0], [0, 5, 0], [0, -1, 0]])
    img_enhance = cv2.filter2D(imgName,-1,kernel=kernel)
    mask_blue = cv2.inRange(img_enhance,(100,0,0),(255,140,140))
    img_hsv = cv2.cvtColor(imgName,cv2.COLOR_BGR2HSV)
    _,s,_ = cv2.split(img_hsv)
    mask_s = cv2.inRange(s,80,255)
    img_result = mask_blue & mask_s
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (15, 3))
    img_dilate = cv2.dilate(img_result, kernel, 3)   # 膨胀 ，减小车牌空洞
    return img_dilate

def img_pro_green(imgName):
    img_gaus = cv2.GaussianBlur(imgName,(5,5),0)
    mask_green = cv2.inRange(img_gaus,(0,100,0),(140,200,140))
    img_hsv = cv2.cvtColor(imgName,cv2.COLOR_BGR2HSV)
    _,s,_ = cv2.split(img_hsv)
    mask_s = cv2.inRange(s,80,255)
    img_result = mask_green & mask_s
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (10, 3))
    img_dilate = cv2.dilate(img_result, kernel, 3)   # 膨胀 ，减小车牌空洞
    return img_dilate

def img_pro_blue(imgName):
    img_gaus = cv2.GaussianBlur(imgName,(5,5),0)
    mask_blue = cv2.inRange(img_gaus,(100,0,0),(200,140,140))
    img_hsv = cv2.cvtColor(imgName,cv2.COLOR_BGR2HSV)
    _,s,_ = cv2.split(img_hsv)
    mask_s = cv2.inRange(s,80,255)
    img_result = mask_blue & mask_s
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (10, 3))
    img_dilate = cv2.dilate(img_result, kernel, 3)   # 膨胀 ，减小车牌空洞

    return img_dilate

def find_location(img,img_dilate,minArea=1000,ra_screen = True):
    contours,_ = cv2.findContours(img_dilate,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    region = []
    for i in range(len(contours)):
        contour = contours[i]
        area = cv2.contourArea(contour)
        rect = cv2.minAreaRect(contour)
        if(area < minArea):
            continue
        box = np.int32(cv2.boxPoints(rect))
        if(ra_screen):
            width, height = box[1]
            if(height == 0):
                continue
            ratio = float(width) / float(height)
            if(ratio > maxPlateRatio or ratio < minPlateRatio):
                continue
        Xs = [i[0] for i in box]
        Ys = [i[1] for i in box]
        x1 = min(Xs)
        x2 = max(Xs)
        y1 = min(Ys)
        y2 = max(Ys)
        hight = y2 - y1
        width = x2 - x1
        plate = img[y1:y1+hight, x1:x1+width]
        region.append(box)
    img_result = np.copy(img)
    cv2.drawContours(img_result, region, -1, (0, 255, 0), 2)  
    save_path1 = "./result/"+file
    save_path2 = "./plate/"+file
    cv2.imwrite(save_path1,img_result)
    cv2.imwrite(save_path2,plate)
    return img_result



In [42]:
out = []
for file in imgName:
    img = cv2.imread(path+file)
    if(file == "IMG20211116093237.jpg"):
        img_cut = img[400:700,400:700]
        img_pro = img_pro_green(img_cut)
        img_pro = cv2.copyMakeBorder(img_pro,400,740,400,380,cv2.BORDER_REPLICATE)
        img_result = find_location(img,img_pro)
    elif(file == "IMG20211117090817.jpg"):
        img_cut = img[:,0:500]
        img_pro = img_pro_blue(img_cut)
        img_result = find_location(img,img_pro,ra_screen=False)
    else:
        img_pro = img_pro_blue(img)
        if(file == "IMG20211117090625.jpg" or file == "IMG20211117090832.jpg"):
            img_result = find_location(img,img_pro,250)
        elif(file == "IMG20211118142540.jpg"):
            img_result = find_location(img,img_pro,500)
        else:
            img_result = find_location(img,img_pro)
            if((img_result == img).all()):
                img_pro = img_pro_enhance(img)
                img_result = find_location(img,img_pro)
            if((img_result == img).all()):
                img_pro = img_pro_sobel(img)
                img_result = find_location(img,img_pro)
            out.append(img_result)
        
rand = random.randint(0,len(out))
#cv2.imshow("random img",np.array(out[rand]))
#cv2.waitKey(0)    


UnboundLocalError: local variable 'plate' referenced before assignment